In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 3.5MB/s 
     |████████████████████████████████| 2.7MB 29.1MB/s 


In [0]:
import math, sys
from konlpy.tag import Okt

class Filter:

    def __init__(self):
        self.words = set()
        self.word_dict = {}
        self.category_dict = {}

    def split(self, text):
        results = []
        twitter = Okt()
        malist = twitter.pos(text, norm=True, stem=True)
        # 실습 2
        # 아래 for 문을 한줄짜리 for 문으로 바꿔보세요 List Comprehension
        for word in malist:
            if not word[1] in ["Josa", "Eomi","Punctuation"]:
                results.append(word[0])
        # results = [word[0] for word in malist if not word[1] in["Josa","Eomi","Punctuation"]]
        return results


    def inc_word(self, word, category):
        if not category in self.word_dict:
            self.word_dict[category] = {}
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1
        self.words.add(word)

    def inc_category(self, category):
        if not category in self.category_dict:
            self.category_dict[category] = 0
        self.category_dict[category] += 1

    def fit(self, text, category):
        word_list = self.split(text)
        for word in word_list:
            self.inc_word(word,category)
        self.inc_category(category)

    def score(self, words, category):
        score = math.log(self.category_prob(category))
        for word in words:
            score += math.log(self.word_prob(word, category))
        return score

    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize # - 무한대를 변수에 지정
        words = self.split(text)
        score_list = []
        for category in self.category_dict.keys(): # category_dict = {'광고':6, '중요':5, '소셜':4} ->키값은 광고중요소셜
            score = self.score(words, category) #카테고리->키값
            score_list.append((category,score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list

    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0

    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        return category_v / sum_categories

    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1 
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n/d

In [0]:
bf = Filter()

bf.fit("파격 세일 - 오늘까지만 30% 할인",  "광고")
bf.fit("쿠폰 선물 & 무료 배송", "광고")
bf.fit("현대 백화점 세일", "광고")
bf.fit("봄과 함께 찾아온 따뜻한 신제품 소식", "광고")
bf.fit("회원님에게만 추천 드리는 상품", "광고")
bf.fit("인기 제품 기간 한정 세일", "광고")
bf.fit("오늘 일정 확인", "중요")
bf.fit("프로젝트 진행 상황 보고", "중요")
bf.fit("계약 잘 부탁드립니다", "중요")
bf.fit("회의 일정이 등록되었습니다", "중요")
bf.fit("오늘 일정이 없습니다", "중요")
bf.fit("1촌 신청을 기다립니다", "소셜")
bf.fit("1촌이 새로운 게시물을 올렸습니다", "소셜")
bf.fit("강수영님을 아세요?", "소셜")
bf.fit("홍길동님이 회원님의 사진에 좋아요를 눌렀습니다", "소셜")

In [52]:
pre, scorelist = bf.predict("오류오류오류")
print("결과 =", pre)
print(scorelist)

결과 = 중요
[('광고', -9.82498532438117), ('중요', -9.65194452670022), ('소셜', -9.983222520554982)]


In [0]:
# 실습 1 
# Filter Class 분석
# 어떤식으로든 상관없습니다
# 예시)
# bf.fit은 text와 뭐를 변수로 받아서 데이터를 처리하는데 어떤 함수를 호출하고 ~~~ 이런 결과가 된다
# bf.predict 는 ~~~~~ 해서 ~~~~~ 이런 결과를 도출한다 